In [ ]:
# pip install faiss-gpu-cu12

In [ ]:
# pip install langchain-text-splitters

In [ ]:
# pip install langchain-huggingface sentence-transformers

In [ ]:
# pip install langchain-community

In [ ]:
# pip install pypdf

In [ ]:
# pip install jq

In [ ]:
import os
import json
import pandas as pd
import random

In [ ]:
from typing import List, Optional, Dict

import faiss
import numpy as np
from tqdm import tqdm
from langchain_community.document_loaders import (
    PyPDFLoader,
    TextLoader,
    JSONLoader
)
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.documents import Document

from transformers import AutoTokenizer, AutoModel, AutoModelForCausalLM

import torch
from torch import Tensor
import torch.nn.functional as F

In [ ]:
# pip install matplotlib

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
# main_path = '/content/drive/MyDrive/PRACTICUM_DLE/sprint_7/'
main_path = ''

# 1 EDA

In [ ]:
data_path = main_path + 'nlp_s3_project/'

data_json_path = data_path + 'arxiv-metadata-s.json'
data_csv_path = data_path + 'test_sample.csv'

data_sample_path = data_path + 'sample_data.json'
data_sample2_path = data_path + 'sample_data2.json'
data_sample3_path = data_path + 'sample_data3.json'
data_sample4_path = data_path + 'sample_data4.json'

In [ ]:
# with open(data_path + 'sample_data.json', 'w') as file:
#   json.dump(train_data[:10000], file)

In [ ]:
with open(data_json_path, 'r') as file:
  train_data = json.load(file)

In [ ]:
with open(data_sample_path, 'r') as file:
  sample_data = json.load(file)

In [ ]:
with open(data_sample2_path, 'r') as file:
  sample_data2 = json.load(file)

In [ ]:
with open(data_sample3_path, 'r') as file:
  sample_data3 = json.load(file)

In [ ]:
with open(data_sample4_path, 'r') as file:
  sample_data4 = json.load(file)

In [ ]:
len(sample_data), len(sample_data2), len(sample_data3), len(sample_data4), 

In [ ]:
df_test = pd.read_csv(data_csv_path)

In [ ]:
df_test.shape

In [ ]:
# print(df_test[:100]['id'].tolist())

In [ ]:
# our_ids = []
# for i, elem in tqdm(enumerate(train_data)):
#     if elem['id'] in df_test[:100]['id'].tolist():
#         # print(i)
#         our_ids.append(i)

In [ ]:
# our_ids = []
# for j, row in tqdm(df_test[:100].iterrows()):
#     for i, elem in (enumerate(train_data)):
#         if elem['id'] == row['id']:
#             our_ids.append(i)

In [ ]:
# sample_data4 = [train_data[i] for i in our_ids] #+ random.sample(train_data, 100)
# len(sample_data4)

In [ ]:
# sample_data4[0]

In [ ]:
# with open(data_path + 'sample_data4.json', 'w') as file:
#   json.dump(sample_data4, file)

In [ ]:
df_test.head()

## 1.1 Train data

In [ ]:
df_train = pd.DataFrame(random.sample(train_data, 10000))

In [ ]:
df_train.head(3)

## title

In [ ]:
df_train['title'].apply(lambda x: len(x.split())).hist()

In [ ]:
df_train['title'].str.len().hist()

In [ ]:
df_train['abstract'].apply(lambda x: len(x.split())).hist()

In [ ]:
df_train['abstract'].str.len().hist()

In [ ]:
embedder_name = "Qwen/Qwen3-Embedding-0.6B"

device = "cuda" if torch.cuda.is_available() else "cpu"
emb_tokenizer = AutoTokenizer.from_pretrained(embedder_name)

In [ ]:
rezu = emb_tokenizer(df_train['abstract'].tolist())

In [ ]:
rezu.keys()

In [ ]:
from matplotlib import pyplot as plt

In [ ]:
plt.hist([len(tk) for tk in rezu['input_ids']])

In [ ]:
[len(tk) for tk in rezu['input_ids']]

## submitter

In [ ]:
df_train['submitter'].value_counts()

## tags

In [ ]:
categories_dict = {}
for cats in df_train['categories']:
  for cat in cats.split():
    if cat in categories_dict:
      categories_dict[cat] += 1
    else:
      categories_dict[cat] = 1

In [ ]:
df_cats = pd.DataFrame([[k, v] for k, v in categories_dict.items()], columns=['category', 'count']).sort_values('count', ascending=False).reset_index(drop=True)
df_cats

In [ ]:
df_train['categories'].apply(lambda x: len(x.split())).value_counts().reset_index()

## update_date

In [ ]:
pd.to_datetime(df_train['update_date']).hist()

## versions

In [ ]:
df_train['versions'].apply(lambda x: len(x)).value_counts()

## authors_parsed

In [ ]:
df_train.head(2)

In [ ]:
df_train['authors_parsed'].apply(lambda x: len(x)).value_counts().reset_index()

## 1.2 Test data

In [ ]:
df_test.head()

In [ ]:
df_test.loc[0, 'query']

In [ ]:
df_test.loc[0, 'abstract']

In [ ]:
df_test['abstract'].apply(lambda x: len(x.split())).hist()

In [ ]:
df_test['query'].apply(lambda x: len(x.split())).hist()

# Выводы по EDA

1. как вы будете решать задачу;
- Сделаем RAG+LLM
2. какие подходы к поиску примените и почему;
- Графовая БД index какой-то
3. какие модели выберете и от каких ограничений будете отталкиваться;
- Как в примерах
4. из чего будет состоять ваша система.
- Загрузчик данных
- Ембеддер данных
- БДшка векторная и индекс
- функция поиска релевантных данных к запросу
- функция ранжирования релеввантных
- Формирование комби запроса с учетом релевантных
- Инференс с ЛЛМ
- расчет метрики

# 2 Этап 2. Реализация retrieval-системы

In [ ]:
# from langchain_core.documents import Document
# def load_from_list_of_dicts(data: list[dict]) -> list[Document]:
#     """
#     Converts a list of dictionaries into a list of LangChain Document objects.

#     Each dictionary is expected to have 'page_content' and optional 'metadata' keys.
#     """
#     documents = []
#     for item in data:
#         page_content = item.get('title') + '\n' + item.get('abstract')
#         metadata = {'source': item.get('categories')}
#         # Create a Document object
#         doc = Document(page_content=page_content, metadata=metadata)
#         documents.append(doc)
#     return documents

In [ ]:
def metadata_func(record: dict, metadata: dict) -> dict:
    # print(f'record = {record}')
    # print(f'record.keys() = {record.keys()}')
    metadata["categories"] = record["categories"]
    metadata["id"] = record["id"]
    return metadata

In [ ]:
class RAG:

    def __init__(
        self,
        # embedder_name: str = "intfloat/e5-base-v2",
        # reranker_name: str = "cross-encoder/ms-marco-MiniLM-L-12-v2",

        embedder_name: str = "Qwen/Qwen3-Embedding-0.6B",
        reranker_name: str = "Qwen/Qwen3-Reranker-0.6B",

        # chunk_size: int = 500,
        # chunk_overlap: int = 125,
        device: Optional[str] = None,
    ):
        self.device = device or ("cuda"
                                 if torch.cuda.is_available() else "cpu")
        self.emb_tokenizer = AutoTokenizer.from_pretrained(embedder_name)
        self.embedder = AutoModel.from_pretrained(embedder_name).to(
            self.device)
        self.embedder.eval()

        self.rr_tokenizer = AutoTokenizer.from_pretrained(
            reranker_name,
            padding_side='left')
        self.reranker = AutoModelForCausalLM.from_pretrained(
            reranker_name).to(self.device)
        self.reranker.eval()

        # self.text_splitter = RecursiveCharacterTextSplitter(
        #     chunk_size=chunk_size,
        #     chunk_overlap=chunk_overlap,
        # )
        self.index = None
        self.doc_store = []

        self.max_length = 8192
        # self.max_length = 512
        self.token_false_id = self.rr_tokenizer.convert_tokens_to_ids("no")
        self.token_true_id = self.rr_tokenizer.convert_tokens_to_ids("yes")
        prefix = "<|im_start|>system\nJudge whether the Document meets the requirements based on the Query and the Instruct provided. Note that the answer can only be \"yes\" or \"no\".<|im_end|>\n<|im_start|>user\n"
        suffix = "<|im_end|>\n<|im_start|>assistant\n<think>\n\n</think>\n\n"
        self.prefix_tokens = self.rr_tokenizer.encode(prefix,
                                                      add_special_tokens=False)
        self.suffix_tokens = self.rr_tokenizer.encode(suffix,
                                                      add_special_tokens=False)

    def _generate_embeddings(self, texts: List[str]) -> np.ndarray:
        inputs = self.emb_tokenizer(
            texts,
            padding=True,
            truncation=True,
            return_tensors="pt",
            max_length=self.max_length,
        ).to(self.device)

        with torch.no_grad():
            outputs = self.embedder(**inputs)

        inputs.to("cpu")
        embeddings = self.last_token_pool(outputs.last_hidden_state,
                                          inputs.attention_mask).cpu()
        return F.normalize(embeddings, p=2, dim=1).numpy()

    @staticmethod
    def last_token_pool(last_hidden_states: Tensor,
                        attention_mask: Tensor) -> Tensor:
        left_padding = (attention_mask[:, -1].sum() == attention_mask.shape[0])
        if left_padding:
            return last_hidden_states[:, -1]
        else:
            sequence_lengths = attention_mask.sum(dim=1) - 1
            batch_size = last_hidden_states.shape[0]
            return last_hidden_states[
                torch.arange(batch_size, device=last_hidden_states.device),
                sequence_lengths]

    # Define the metadata extraction function.


    def load_and_process_file(self, file_path: str) -> List[Document]:
        """Загрузка и экстракция данных из файлов"""
        ext = os.path.splitext(file_path)[1]
        if ext == ".json":
            loader = JSONLoader(
              file_path=file_path,
              jq_schema='.[]',
              content_key="abstract",
              text_content=True,
              # is_content_key_jq_parsable=True,
              metadata_func=metadata_func
            )
            # loader = JSONLoader(
            #   file_path=file_path,
            #   jq_schema='.[] | {combined_content: (.title + ". " + .abstract), categories: .categories, id: .id}',
            #   content_key="combined_content",
            #   text_content=True,
            #   # is_content_key_jq_parsable=True,
            #   metadata_func=metadata_func
            # )
        else:
            raise ValueError(f"Unsupported file type: {ext}")

        docs = loader.load()

        # return self.text_splitter.split_documents(docs)
        return docs


    def build_index(self, file_paths: List[str], batch_size: int = 4) -> None:
        """Строим индекс FAISS"""
        all_docs = []
        for path in file_paths:
            all_docs.extend(self.load_and_process_file(path))
        self.doc_store = all_docs

        # Вычислим numpy-эмбеддинги по батчам
        embeddings = []
        for i in tqdm(range(0, len(all_docs), batch_size)):
            batch = [doc.page_content for doc in all_docs[i:i + batch_size]]
            embeddings.append(self._generate_embeddings(batch))

        embeddings = np.concatenate(embeddings)

        # Инициализируем индекс
        self.index = faiss.IndexFlatL2(embeddings.shape[1])
        self.index.add(embeddings)

    @staticmethod
    def get_detailed_instruct(task_description: str, query: str):
        return f'Instruct: {task_description}\nQuery:{query}'

    @staticmethod
    def format_reranker_instruction(query, doc, instruction=None):
        if instruction is None:
            instruction = 'Given a web search query, retrieve relevant passages that answer the query'
        output = "<Instruct>: {instruction}\n<Query>: {query}\n<Document>: {doc}".format(
            instruction=instruction, query=query, doc=doc)
        return output

    def process_inputs(self, pairs):
        """Обработка данных для реранкера"""
        inputs = self.rr_tokenizer(pairs,
                                   padding=False,
                                   truncation='longest_first',
                                   return_attention_mask=False,
                                   max_length=self.max_length -
                                   len(self.prefix_tokens) -
                                   len(self.suffix_tokens))
        for i, ele in enumerate(inputs['input_ids']):
            inputs['input_ids'][
                i] = self.prefix_tokens + ele + self.suffix_tokens
        inputs = self.rr_tokenizer.pad(inputs,
                                       padding=True,
                                       return_tensors="pt",
                                       max_length=self.max_length)

        # переносим тензоры на девайс ранжирующей модели
        for key in inputs:
            inputs[key] = inputs[key].to(self.device)
        return inputs

    def search(self,
               query: str,
               k: int = 5,
               task: str = None):
        if self.index is None:
            raise ValueError("Index not initialized")

        if task is None:
            task = 'Given a web search query, retrieve relevant passages that answer the query'

        query_embedding = self._generate_embeddings([query])
        distances, indices = self.index.search(query_embedding, k)
        return distances, indices

    @torch.no_grad()
    def compute_logits(self, inputs):
        batch_scores = self.reranker(**inputs).logits[:, -1, :]
        true_vector = batch_scores[:, self.token_true_id]
        false_vector = batch_scores[:, self.token_false_id]
        batch_scores = torch.stack([false_vector, true_vector], dim=1)
        batch_scores = torch.nn.functional.log_softmax(batch_scores, dim=1)
        scores = batch_scores[:, 1].exp().tolist()
        return scores

    def rerank(self, query: str, documents: List[str], batch_size=1):
        pairs = []
        for d in documents:
            pairs.append(self.format_reranker_instruction(query, d))

        scores = []
        for i in range(0, len(pairs), batch_size):

            inputs = self.process_inputs(pairs[i:i + batch_size])
            sc = self.compute_logits(inputs)
            scores.extend(sc)
        return scores

In [ ]:
def find_index_next(my_list, target_value, not_found_value):
    return next((i + 1 for i, x in enumerate(my_list) if x == target_value), not_found_value)

In [ ]:
# # освободим место на GPU
# torch.cuda.empty_cache()
# rag.embedder.to("cpu")
# rag.reranker.to("cpu")

In [ ]:
rag = RAG(device="cuda")

In [ ]:
rag.build_index(
    [
        data_json_path,

        # data_sample_path,
        # data_sample2_path,
        # data_sample4_path,
        
    ],
    batch_size=1
)

In [ ]:
test_check_idx = 4

In [ ]:
q = df_test.loc[test_check_idx, 'query']
answer = df_test.loc[test_check_idx, 'abstract']
id_true = df_test.loc[test_check_idx, 'id']
q, id_true, answer

In [ ]:
for i, elem in enumerate(sample_data4):
    if elem['id'] == id_true:
        print(i, elem['id'])
        break

In [ ]:
k_cands = 5

In [ ]:
_, I = rag.search(q, k=k_cands)

In [ ]:
candidates = [rag.doc_store[i].page_content for i in I[0]]

In [ ]:
I[0]

In [ ]:
rag.doc_store[39]

In [ ]:
relevant_flags = [rag.doc_store[i].metadata['id'] == id_true for i in I[0]]
find_index_next(relevant_flags, True, np.inf)

In [ ]:
k_ranked = 100

In [ ]:
candidates[0]

In [ ]:
scores = rag.rerank(q, candidates)
array = np.array(scores)
indices = np.argsort(array)[::-1][:k_ranked]

In [ ]:
indices

In [ ]:
ranked_ids = [int(I[0][i]) for i in indices]
ranked_ids

In [ ]:
relevant_flags = [rag.doc_store[i].metadata['id'] == id_true for i in ranked_ids]

In [ ]:
find_index_next(relevant_flags, True, np.inf)

In [ ]:
K_mrr = 5
N_cands_for_rerank = 5

mrr5_list = []
for j, row in tqdm(df_test.iterrows()):
  # print(index, row['id'])
  q = row['query']
  id_true = row['id']


  _, I = rag.search(q, k=N_cands_for_rerank)
  candidates = [rag.doc_store[i].page_content for i in I[0]]
  scores = rag.rerank(q, candidates)
  array = np.array(scores)
  indices = np.argsort(array)[::-1][:K_mrr]
  ranked_ids = [int(I[0][i]) for i in indices]

  relevant_flags = [rag.doc_store[i].metadata['id'] == id_true for i in ranked_ids]   # i in I[0]]
  relev_index = find_index_next(relevant_flags, True, np.inf)

  mrr5_step = 1 / relev_index
  if mrr5_step > 0 :
    print(f'j={j}, relev_index = {relev_index}, mrr5_step={mrr5_step}')
  mrr5_list.append(mrr5_step)

In [ ]:
MRR5 = sum(mrr5_list) / len(mrr5_list)
MRR5